In [31]:
import math
from math import e
import glob
import matplotlib.pyplot as plt
from IPython.display import clear_output
from scipy.io import loadmat

In [110]:
import json
import time
import random
import numpy as np
import pandas as pd
import scipy.stats as st

class DecisionModeler:
    
    def __init__(self):
        pass
    
    def p_cc2a(self, dp, k1, k2):
        a_given_a = st.norm.cdf(k1 + dp/2)
        b_given_a = 1 - st.norm.cdf(k2 + dp/2)
        a_given_b = st.norm.cdf(k1 - dp/2)
        b_given_b = 1 - st.norm.cdf(k2 - dp/2)
        
        pAA = a_given_a*b_given_a + b_given_a*a_given_a
        pBB = a_given_b*b_given_b + b_given_b*a_given_b
        pAB = a_given_a*b_given_b + b_given_a*a_given_b
        pBA = a_given_b*b_given_a + b_given_b*a_given_a

        return pAA, pBB, pAB, pBA
    
    def p_df2(self, dp, k1, k2):
        pAA = pBB = st.norm.cdf(-k1/math.sqrt(2)) + st.norm.cdf(-k2/math.sqrt(2))
        pAB = st.norm.cdf((-dp - k2)/math.sqrt(2)) + st.norm.cdf((dp - k1)/math.sqrt(2))
        pBA = st.norm.cdf((-dp - k1)/math.sqrt(2)) + st.norm.cdf((dp - k2)/math.sqrt(2))
        
        return pAA, pBB, pAB, pBA
    
    def p_lr2(self, dp, b1, b2):
        if b1 > 1:
            pAB = 1 - 2*(1 - st.norm.cdf(np.log(b1)/dp - dp/2))*(1 - st.norm.cdf(np.log(b1)/dp + dp/2))
            p_false_b1 = 1 - ((1 - st.norm.cdf(np.log(b1)/dp - dp/2))**2 + (1 - st.norm.cdf(np.log(b1)/dp + dp/2))**2)
        else:
            pAB = ((1 - st.norm.cdf(np.log(1/b1)/dp - dp/2))**2 + (1 - st.norm.cdf(np.log(1/b1)/dp + dp/2))**2)
            p_false_b1 = 2*(1 - st.norm.cdf(np.log(1/b1)/dp - dp/2))*(1 - st.norm.cdf(np.log(1/b1)/dp + dp/2))
            
        if b2 > 1:
            pBA = 1 - 2*(1 - st.norm.cdf(np.log(b2)/dp - dp/2))*(1 - st.norm.cdf(np.log(b2)/dp + dp/2))
            p_false_b2 = 1 - ((1 - st.norm.cdf(np.log(b2)/dp - dp/2))**2 + (1 - st.norm.cdf(np.log(b2)/dp + dp/2))**2)
        else:
            pBA = ((1 - st.norm.cdf(np.log(1/b2)/dp - dp/2))**2 + (1 - st.norm.cdf(np.log(1/b2)/dp + dp/2))**2)
            p_false_b2 = 2*(1 - st.norm.cdf(np.log(1/b2)/dp - dp/2))*(1 - st.norm.cdf(np.log(1/b2)/dp + dp/2))
            
        pAA = pBB = (p_false_b1 + p_false_b2)/2
        return pAA, pBB, pAB, pBA
    
    def generate_1_param_model(self, dp_vals, k_vals, p_function, filepath):
        model = dict()
        
        # Variables used to keep track of progress
        counter = progress = 0
        total = len(dp_vals)
        
        # Calculate p-value tuples for each d' value
        for dp in dp_vals:
            dp_list = set()
            for k in k_vals:
                dp_list.add(p_function(dp, k, k))
            model[dp] = list(dp_list)
            
            counter += 1
            if counter/total >= progress+0.1:
                progress += 0.1
                print(f'{round(progress*100, 2)}% evaluated.')
            
        # Write data to json file
        with open(filepath, 'w') as file:
            json.dump(model, file)
            print('Successfully written data to file.')

    def generate_2_param_model(self, dp_vals, k_vals, p_function, filepath, model_type=""):
        model = dict()
        model_type = model_type.lower()
        
        # Variables used to keep track of progress
        counter = progress = 0
        total = len(dp_vals)
        
        # Calculate p-value tuples for each d' value
        for dp in dp_vals:
            dp_list = set()
            for k1 in k_vals:
                for k2 in k_vals:
                    # Ensure k2 value is appropriate depending on model
                    if (model_type=="cc2a" or model_type=="lr2") and k2 < k1:
                        continue
                    if model_type=="df2" and k2 < -k1:
                        continue
                    # Calculate p-value tuple and add to current d' list
                    dp_list.add(p_function(dp, k1, k2))
            model[dp] = list(dp_list)
            
            counter += 1
            if counter/total >= progress+0.1:
                progress += 0.1
                print(f'{round(progress*100, 2)}% evaluated.')
            
        # Write data to json file
        with open(filepath, 'w') as file:
            json.dump(model, file)
            print('Successfully written data to file.')
    
    def get_distance(self, pHuman, pModel):
        # 2d distance: (pAA_h + pBB_h - pAA_m - pBB_m)**2 + ...
        dist_2d = math.sqrt((pHuman[0] + pHuman[1] - pModel[0] - pModel[1])**2 + (pHuman[2] + pHuman[3] - pModel[2] - pModel[3])**2)
        return np.sqrt(np.sum((pHuman - pModel)**2)) + dist_2d
    
    def get_chi_squared(self, pHuman, pModel):
        
        # Number of trials in each block
        n = 18
        
        # See Petrov equation 5
        '''
        chi_squared = 0
        for pH, pM in zip(pHuman, pModel):
            p_avg = (pH + pM) / 2
            if p_avg == 0:
                return np.inf
            chi_squared += n*((pH-p_avg)**2 + (pM-p_avg)**2)/p_avg/(1-p_avg)
        '''
        
        # See Collett equation 2.18
        #chi_squared = sum([n*(pH-pM)**2/pM + n*((1-pH)-(1-pM))**2/(1-pM) for pH, pM in zip(pHuman, pModel)])
        chi_squared = [n*(pH-pM)**2/pM for pH, pM in zip(pHuman, pModel)]
        chi_squared.extend([n*((1-pH)-(1-pM))**2/(1-pM) for pH, pM in zip(pHuman, pModel)])
                
        return chi_squared
    
    def search_dp(self, pHuman_vals, model_input, output_prefix="sample", default_df=None):
        if default_df is None:
            default_df = len(pHuman_vals)
        
        # Variables used to track best fit d' with distance, chi-squared, and closest points
        min_dist = np.inf
        min_chi_squared = None
        best_dp = 0
        best_pts = None
        best_cs = None
        
        df = pd.DataFrame(columns=['dp', 'distance', 'chi-square', 'df', 'p-value'])
        with open(model_input, 'rb') as file:
            model = json.load(file)
            
            # Instantiate variables for individual d' values
            dist_arr = np.empty(len(pHuman_vals))
            chi_squared_arr = np.empty(len(pHuman_vals))
            
            # Variables used to keep track of progress
            start = time.time()
            counter = progress = 0
            total = len(model.keys())
            
            # Try fit with each available d' value
            for dp in model.keys():
                # Reset variables for current d'
                dist_arr.fill(np.inf)
                chi_squared_arr.fill(np.inf)
                best_dp_pts = np.empty((len(pHuman_vals), 4))
                best_dp_cs = np.empty((len(pHuman_vals), 8))

                for pModel in model[dp]:
                    # Try p-value tuple fit with each human data point
                    for index, pHuman in enumerate(pHuman_vals):
                        dist = self.get_distance(pHuman, pModel)
                        chi_squared = self.get_chi_squared(pHuman, pModel)
                        
                        # Check if minimal distance for current pModel, pHuman pair
                        if dist < dist_arr[index]:
                            dist_arr[index] = dist
                            chi_squared_arr[index] = sum(chi_squared)
                            
                            best_dp_pts[index] = pModel
                            best_dp_cs[index] = chi_squared
                
                # Update dataframe with data for current d'
                curr_chi_square = sum([c for c in chi_squared_arr if c != np.inf])
                curr_df = default_df - 2*np.count_nonzero(chi_squared_arr==np.inf)
                df.loc[len(df)] = [dp, sum(dist_arr), curr_chi_square, curr_df, 1-st.chi2.cdf(curr_chi_square, curr_df)]
                
                # Update best fit variables if distance is new minimum
                if sum(dist_arr) < min_dist:
                    best_pts = best_dp_pts
                    best_cs = best_dp_cs
                    
                    min_dist = sum(dist_arr)
                    min_chi_squared = chi_squared_arr
                    best_dp = float(dp)
                    
                counter += 1
                if counter/total >= progress+0.1:
                    progress += 0.1
                    print(f'{round(progress*100, 2)}% evaluated.')
        
        # Write data to files
        df = df.sort_values(by=['distance'])
        df = df.reset_index(drop=True)
        df.to_csv(f'{output_prefix}_fit.csv')
        
        pts_df = pd.DataFrame(columns=['pAA', 'pBB', 'pAB', 'pBA'])
        for p in best_pts:
            pts_df.loc[len(pts_df)] = p
        pts_df.to_csv(f'{output_prefix}_points.csv')
        
        cs = pd.DataFrame(columns=['AA','BB','AB','BA','AA_c','BB_c','AB_c','BA_c'])
        for p in best_cs:
            cs.loc[len(cs)] = p
        cs.to_csv(f'{output_prefix}_cs.csv')
        
        print(f'Completed in {round((time.time()-start)/60, 2)} minutes.')
        return best_dp, sum(chi_squared_arr), best_pts
                                       

# Model Generation
Recommended criterion ranges: 
<ul>
    <li>CC: [-3,3]</li>
    <li>DF: [0,6]</li>
    <li>LR: [e**-4,e**4]</li>
    <li>RC: [0,2]</li>
</ul>

In [112]:
modeler = DecisionModeler()

dp_vals = np.linspace(0, 2.5, 11)
#k_vals = np.linspace(-4, 4, 51)

k_vals = np.logspace(-4, 4, 51, base=e)
index = np.argwhere(dp_vals==0)
dp_vals = np.delete(dp_vals, index)

modeler.generate_2_param_model(dp_vals, k_vals, modeler.p_lr2, 'lr2_xs.json', model_type="lr2")

10.0% evaluated.
20.0% evaluated.
30.0% evaluated.
40.0% evaluated.
50.0% evaluated.
60.0% evaluated.
70.0% evaluated.
80.0% evaluated.
90.0% evaluated.
Successfully written data to file.


In [73]:
modeler = DecisionModeler()

dp_vals = np.linspace(0, 2.5, 11)
k_vals = np.linspace(0, 6, 51)

modeler.generate_2_param_model(dp_vals, k_vals, modeler.p_df2, 'df2_xs.json', model_type="df2")

10.0% evaluated.
20.0% evaluated.
30.0% evaluated.
40.0% evaluated.
50.0% evaluated.
60.0% evaluated.
70.0% evaluated.
80.0% evaluated.
90.0% evaluated.
100.0% evaluated.
Successfully written data to file.


# Exhaustive Search

In [94]:
modeler = DecisionModeler()

types = ['AA', 'BB', 'AB', 'BA']
p_data = pd.read_csv('p_data.csv', index_col="Subject")
ratings_threshold = 1.0

files = glob.glob('raw/8/*.mat')
for f in files:
    
    dat = loadmat(f)
    subject = f[max(f.rfind('/'), f.rfind('\\'))+1:f.find('.')]
    
    print(f'Currently checking subject {subject}.')
    
    num_ratings = 0
    for i in range(1, 6):
        if p_data.loc[subject][f'PF{i}'] <= ratings_threshold:
            num_ratings += 1
    
    print(f'Number of ratings to be used: {num_ratings}')
    
    points = []
    for t in types:
        points.append(np.concatenate([dat[f'p_{t}_{n+1}Diff'][0] for n in range(num_ratings)]))
    points = np.array(points).T
    
    best_dp, _, model_pts = modeler.search_dp(points, model_input='lr2_xs.json', output_prefix=f'models/lr2/{subject}', default_df=2*len(points))
    clear_output()
    

# Data Output

In [ ]:
files = glob.glob('models/df2/*fit.csv')
model = 'DF2'

explainable = pd.read_csv('subject_models.csv', index_col='Subject')
df = pd.DataFrame(columns=['subject', 'dp', 'distance', 'chi-square', 'df', 'p-value', 'dp2', 'distance2', 'chi-square2', 'df2', 'p-value2', 'explainable'])
for f in files:
    subject = f[max(f.rfind('/'), f.rfind('\\'))+1:f.find('_')]
    
    curr = pd.read_csv(f, index_col=0)
    data = [subject, *curr.iloc[0].tolist(), *curr.iloc[1].tolist(), explainable.loc[subject][model]]
    df.loc[len(df)] = data
    
df = df.sort_values(by=['p-value', 'chi-square'], ascending=[True, False])
df['rejected'] = df['p-value']<0.05
df = df.reset_index(drop=True)

df.to_csv('df2_8deg.csv')